In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
import datetime

## A partir de features elegidos con DECISION TREE

In [2]:
train_features = pd.read_csv('features/feature_selection/features_by_random_forest.csv').drop(columns=['Unnamed: 0'])
train = pd.read_csv("original_data/train.csv", dtype = {"target": int}, encoding='UTF_8')
train = train[['id','target']]
train_features = train_features.merge(train, on='id')
train_features.head()

,id,K_storm,K_bomb,K_typhoon,K_wildfire,K_fire,K_derailment,K_suicide bomb,K_outbreak,K_earthquake,...,tag_street,tag_plan,tag_charged,tag_photo_y,tag_bad,tag_level,tag_guy,tag_hail,tag_flooding,target
0,1,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
1,4,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
2,5,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
3,6,0,0,0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
4,7,0,0,0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.570573,0.0,0.0,0.0,0.0,0.0,1


In [3]:
test_features = pd.read_csv('feature_selection_on_test_RF').drop(columns='Unnamed: 0').fillna(0)

In [4]:
test_features.head()

,id,K_storm,K_bomb,K_typhoon,K_wildfire,K_fire,K_derailment,K_suicide bomb,K_outbreak,K_earthquake,...,tag_love,tag_street,tag_plan,tag_charged,tag_photo_y,tag_bad,tag_level,tag_guy,tag_hail,tag_flooding
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0,0,0,0,0,0,0,1,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0,0,0,1,0,0,0,0,...,0.0,0.74681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,0,0,0,1,1,0,0,0,0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11,0,0,1,0,0,0,0,0,0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#####################################

In [5]:
def save_prediction_accuracy(modelo,x_train,y_train,x_test,y_test):
    with open("prediction_history.csv", "a") as myfile:
        predicted = modelo.predict(x_train)
        train_prediction = str(metrics.accuracy_score(y_train,predicted))
        print('Score para x_train: '+ train_prediction)
        predicted = modelo.predict(x_test)
        test_prediction = str(metrics.accuracy_score(y_test,predicted))
        print('Score para x_test: '+ test_prediction)
        params = str(modelo)
        print('Hiperparametros: '+ str(modelo))
        myfile.write(params+','+test_prediction+","+train_prediction+","+str(datetime.datetime.now())+"\n")

In [6]:
def save_prediction(model, test, name):
    df_prediccion = test[['id']].copy()
    predicted = model.predict(test.drop(columns='id'))
    df_prediccion['target'] = predicted
    df_prediccion.to_csv('predictions/'+name+str(datetime.datetime.now())+'.csv', index=None)
    return

In [7]:
train_df = train_features.drop(columns = ['target'])
train_df = train_df.fillna(0)

#Split the CountVector vectorized data into train and test datasets for model training and testing
X_train, X_test, y_train, y_test =train_test_split(train_df,train_features.target,test_size=0.2,random_state=2020)

## Estimación de accuracy

aca hay mas cosas interesantes https://www.ritchieng.com/machine-learning-evaluate-classification-model/

In [8]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #errors = abs(predictions - test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    accuracy = round(accuracy_score(test_labels,predictions)*100)
    print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

In [9]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [10]:
def fit_and_predict(model,X_train,y_train,X_test,y_test):    
    clf = model
    clf.fit(X_train, y_train)
    predictions=clf.predict(X_test)
    confusion_matrix(y_test,predictions)
    conf = metrics.confusion_matrix(y_test, predictions)
    conf = conf.astype('float') / conf.sum(axis=1)[:, np.newaxis]
    print_cm(conf, ['true','false'])
    print('-'*50)
    print(classification_report(y_test,predictions))
    print('-'*50)
    print("{}" .format(model))
    print('-'*50)
    print('Accuracy of classifier on training set:{}%'.format(round(clf.score(X_train, y_train)*100)))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(round(accuracy_score(y_test,predictions)*100)))
    print('*'*50)

In [11]:
models=[LogisticRegression(C=1.0,max_iter=5000),SVC(),MultinomialNB(),DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors=5),RandomForestClassifier(),GaussianNB(),SGDClassifier(loss="hinge", penalty="l2", max_iter=500)]

In [12]:
X_test = X_test.drop(columns='id')
X_train = X_train.drop(columns='id')

In [13]:
for model in models:
    fit_and_predict(model,X_train, y_train,X_test,y_test)

           true false 
     true   0.8   0.2 
    false   0.4   0.6 
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.72      0.84      0.78       849
           1       0.75      0.58      0.66       674

    accuracy                           0.73      1523
   macro avg       0.73      0.71      0.72      1523
weighted avg       0.73      0.73      0.72      1523

--------------------------------------------------
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
--------------------------------------------------
Accuracy of classifier on training set:77%
--------------------------------------------------
Accuracy of classifier on test

## Estimación de accuracy con Cross-Validation ?

In [14]:
def fit_and_predict_cross_val(model,X_train,y_train,X_test,y_test):
    print("{}" .format(model))
    print('-'*50)
    print('Accuracy of classifier on training set:{}%'.format(round(cross_val_score(model, X_train, y_train, scoring='accuracy', cv = 10).mean()*100)))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(round(cross_val_score(model, X_test, y_test, scoring='accuracy', cv = 10).mean()*100)))
    print('*'*50)

In [15]:
for model in models:
    fit_and_predict_cross_val(model,X_train, y_train,X_test,y_test)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
--------------------------------------------------
Accuracy of classifier on training set:75%
--------------------------------------------------
Accuracy of classifier on test set:73%
**************************************************
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
--------------------------------------------------
Accuracy of classifier on training set:65%
--------------------------------------------------
Accuracy of classifier on test set:62%
****

## Random Forest

### Random Search

In [16]:
from sklearn.model_selection import RandomizedSearchCV # Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = list(np.linspace(1, 200, 10, endpoint=True))
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = np.linspace(0.0, 4.0, 10, endpoint=True)
                              
# Minimum number of samples required at each leaf node
min_samples_leaf = np.arange(0.0, 0.5,0.05)
                              
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
                              
random_grid = {"criterion": ["entropy","gini"],
                'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'criterion': ['entropy', 'gini'], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [1.0, 23.11111111111111, 45.22222222222222, 67.33333333333333, 89.44444444444444, 111.55555555555556, 133.66666666666666, 155.77777777777777, 177.88888888888889, 200.0, None], 'min_samples_split': array([0.        , 0.44444444, 0.88888889, 1.33333333, 1.77777778,
       2.22222222, 2.66666667, 3.11111111, 3.55555556, 4.        ]), 'min_samples_leaf': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45]), 'bootstrap': [True, False]}


In [17]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42,n_jobs=-1)# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   30.2s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [18]:
rf_random.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 200.0,
 'max_features': 'auto',
 'min_samples_leaf': 0.15000000000000002,
 'min_samples_split': 0.8888888888888888,
 'n_estimators': 1000}

In [19]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Model Performance
Accuracy = 72.00%.


In [20]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

Model Performance
Accuracy = 62.00%.


In [21]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of -13.89%.


In [22]:
best_random

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=200.0, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=0.15000000000000002,
                       min_samples_split=0.8888888888888888,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
save_prediction_accuracy(best_random, X_train, y_train, X_test, y_test)

Score para x_train: 0.625615763546798
Score para x_test: 0.6152330925804333
Hiperparametros: RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=200.0, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=0.15000000000000002,
                       min_samples_split=0.8888888888888888,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


## LIGHT GBM

In [24]:
import lightgbm as lgb #conda install -c conda-forge lightgbm

params_LG = {'objective':['binary'],
                             'learning_rate':[0.005,0.01,0.05,0.1,0.3],
                             'n_estimators':np.arange(25,200,15),
                             'num_leaves': np.arange(24, 45,3),
                             'feature_fraction': np.arange(0.1, 0.91, 0.2),   
                             'bagging_fraction': np.arange(0.8, 1.01, 0.1),
                             'max_depth': np.arange(3, 12, 1),
                            #{'lambda_l1': np.arange(0, 5)}, # Restaba mucho
                             'lambda_l2': np.arange(0, 3),
                             'min_split_gain': [0.001, 0.01, 0.1],
                             'min_child_weight': [1e-05]+np.arange(5, 11)}
                             

In [25]:
rf = lgb.LGBMClassifier()
rf_LG = RandomizedSearchCV(estimator = rf, param_distributions = params_LG, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs=-1)# Fit the random search model
rf_LG.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.0s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.1, max_depth=-1,
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=100, n_jobs=-1,
                                            num_leaves=31, objective=None,
                                            random_state=None, reg_alpha=0.0,
                                            reg_lambda=0.0, sile...
                                        'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11]),
                           

In [26]:
rf_LG.best_params_

{'bagging_fraction': 0.8,
 'feature_fraction': 0.7000000000000001,
 'lambda_l2': 1,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 6.00001,
 'min_split_gain': 0.01,
 'n_estimators': 100,
 'num_leaves': 27,
 'objective': 'binary'}

In [27]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Model Performance
Accuracy = 72.00%.


In [28]:
best_LG = rf_LG.best_estimator_
random_accuracy = evaluate(best_LG, X_test, y_test)

Model Performance
Accuracy = 75.00%.


In [29]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of 4.17%.


In [30]:
save_prediction_accuracy(best_LG, X_train, y_train, X_test, y_test)

Score para x_train: 0.819047619047619
Score para x_test: 0.7485226526592252
Hiperparametros: LGBMClassifier(bagging_fraction=0.8, boosting_type='gbdt', class_weight=None,
               colsample_bytree=1.0, feature_fraction=0.7000000000000001,
               importance_type='split', lambda_l2=1, learning_rate=0.1,
               max_depth=10, min_child_samples=20, min_child_weight=6.00001,
               min_split_gain=0.01, n_estimators=100, n_jobs=-1, num_leaves=27,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


## NEURONAL NETWORKS

In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

params_MLPC = {'hidden_layer_sizes':[(n,n) for n in range(1,30)],
                              'activation':['identity', 'logistic', 'tanh', 'relu'],
                              'alpha':[1e-06,1e-05,1e-04,1e-03,1e-02,1e-01,1],
                              'beta_1':np.arange(0.0,1,0.01),
                              'beta_2':np.arange(0.0,1,0.01),
                              'early_stopping':[True, False],
                              'epsilon':[1e-07,1e-08,1e-06, 1e-09, 1e-010, 1e-11],
                              'learning_rate':['constant', 'adaptive'],
                              'solver':['adam', 'lbfgs', 'sgd'],
                              'validation_fraction':np.arange(0.15,0.5,0.01),
                               'max_iter':[200,300,400],
                             }

In [32]:
rf = MLPClassifier()
rf_MLP = RandomizedSearchCV(estimator = rf, param_distributions = params_MLPC, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs=-1)# Fit the random search model
rf_MLP.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.4min finished
/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_fun=15000, max_iter=200,
                                           momentum=0.9, n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           random...
                                        'max_iter': [200, 300, 400],
                                        'solver': ['adam', 'lbfgs', 'sgd'],
                                  

In [33]:
rf_MLP.best_params_

{'activation': 'tanh',
 'alpha': 1e-06,
 'beta_1': 0.98,
 'beta_2': 0.8200000000000001,
 'early_stopping': False,
 'epsilon': 1e-10,
 'hidden_layer_sizes': (6, 6),
 'learning_rate': 'constant',
 'max_iter': 400,
 'solver': 'lbfgs',
 'validation_fraction': 0.18000000000000002}

In [35]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Model Performance
Accuracy = 72.00%.


In [36]:
best_MLP = rf_random.best_estimator_
random_accuracy = evaluate(best_MLP, X_test, y_test)

Model Performance
Accuracy = 62.00%.


In [37]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of -13.89%.


In [38]:
save_prediction_accuracy(best_MLP, X_train, y_train, X_test, y_test)

Score para x_train: 0.625615763546798
Score para x_test: 0.6152330925804333
Hiperparametros: RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=200.0, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=0.15000000000000002,
                       min_samples_split=0.8888888888888888,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


## GRADIENT BOOSTING

In [39]:
from sklearn.ensemble import GradientBoostingClassifier as GBC 
params_GBC = {
    'max_features':['auto', 'sqrt', 'log2'],
                             'max_leaf_nodes': [None,1,2,3,4,5,6,8],
                             'min_weight_fraction_leaf': np.linspace(0.0, 0.5, 1, endpoint=True),
                             'learning_rate': np.arange(0.1, 0.5, 0.05),
                             'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
                             'min_samples_leaf': np.arange(0.1, 0.5, 5),  
                             'max_features' : list(range(1,100)),
                             'max_depth': [n for n in range(0,50,5)],
                             'n_estimators': [n for n in range(0,1000,10)],
                             'subsample': np.arange(0.3, 1,0.1),
                             'loss': ['deviance'],
                             'warm_start': [True, False],
                             'presort': ['auto'],

}

In [40]:
rf = GBC()
rf_GBC = RandomizedSearchCV(estimator = rf, param_distributions = params_GBC, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_GBC.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   17.0s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                        criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                    

In [41]:
rf_GBC.best_params_

{'learning_rate': 0.25000000000000006,
 'loss': 'deviance',
 'max_depth': 35,
 'max_features': 52,
 'max_leaf_nodes': None,
 'min_samples_leaf': 0.1,
 'min_samples_split': 0.1,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 190,
 'presort': 'auto',
 'subsample': 0.8000000000000003,
 'warm_start': True}

In [42]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Model Performance
Accuracy = 72.00%.


In [43]:
best_GBC = rf_GBC.best_estimator_
random_accuracy = evaluate(best_GBC, X_test), y_test)

SyntaxError: invalid syntax (<ipython-input-43-ac736983bac4>, line 2)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
save_prediction_accuracy(best_GBC, X_train, y_train, X_test, y_test)

## XGBoost

In [ ]:
import xgboost as xgb #conda install -c conda-forge xgboost 

params_XGB = {
                              'objective': ['binary:logistic'],'learning_rate':np.arange(0.1,0.5,0.1),
                              'n_estimators':np.arange(16,116,15),
                              'scale_pos_weight':np.arange(2,6,1),
                              'max_depth':np.arange(4,12,1),'min_child_weight':np.arange(1,10,1),
                              'gamma':np.arange(0,0.5,0.1),
                              'subsample':np.arange(0.6,1,0.1),'colsample_bytree':np.arange(0.6,0.91,0.05),
                              'colsample_bylevel':np.arange(0.6,0.91,0.05)#,
                             # {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]} # Empeoraba muchísimo con esto, y Luis dijo que no importaba
                   }

In [ ]:
rf = xgb.XGBClassifier()
rf_XGB = RandomizedSearchCV(estimator = rf, param_distributions = params_XGB, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_XGB.fit(X_train, y_train)

In [ ]:
rf_XGB.best_params_

In [ ]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train.drop(columns='id'), y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
save_prediction_accuracy(best_XGB, X_train, y_train, X_test, y_test)

## PREDICCIONES